In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG_stickyActG_stickyStimGL_wslsG_hG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['resp']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[outcomeRew for ([id],[outcomeRew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['resp'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
           1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
           0, 0, 1, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/fullModel_standardInput/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:27:16,018 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:27:16,020 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:27:16,022 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:27:16,023 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:27:16,024 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:27:33,333 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/fullModel_standardInput/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:27:59,400 - DeepRL - DEBUG - global iter =    0 total obj: 12017.9586
2021-07-02 14:28:15,971 - DeepRL - DEBUG - global iter =    1 total obj: 11306.6032
2021-07-02 14:28:32,893 - DeepRL - DEBUG - global iter =    2 total obj: 10724.5079
2021-07-02 14:28:51,999 - DeepRL - DEBUG - global iter =    3 total obj: 10451.3086
2021-07-02 14:29:11,550 - DeepRL - DEBUG - global iter =    4 total obj: 10348.9529
2021-07-02 14:29:31,418 - DeepRL - DEBUG - global iter =    5 total obj: 10270.6580
2021-07-02 14:29:54,297 - DeepRL - DEBUG - global iter =    6 total obj: 10240.6046
2021-07-02 14:30:17,720 - DeepRL - DEBUG - global iter =    7 total obj: 10206.8156
2021-07-02 14:30:41,203 - DeepRL - DEBUG - global iter =    8 total obj: 10188.9394
2021-07-02 14:31:05,733 - DeepRL - DEBUG - global iter =    9 total obj: 10422.5198
2021-07-02 14:31:32,980 - DeepRL - DEBUG - global iter =   10 total obj: 10438.9034
2021-07-02 14:32:01,592 - DeepRL - DEBUG - global iter =   11 total obj: 102

INFO:tensorflow:../results/fullModel_standardInput/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 25,   0,  25, ...,  25,  25,  25],
          [  0, -25,   0, ...,   0,   0,   0],
          [ 25,   0,  25, ...,   0,   0,  25],
          ...,
          [  0, -25,   0, ...,   0,   0,   0],
          [ 25,  25,  25, ...,   0,  25,   0],
          [  0,   0,   0, ..., -25,   0, -25]], dtype=int16),
   'action': array([[0, 1, 0, ..., 0, 1, 0],
          [0, 1, 0, ..., 0, 1, 0],
          [0, 1, 0, ..., 1, 0, 1],
          ...,
          [0, 1, 0, ..., 0, 1, 0],
          [0, 0, 0, ..., 1, 1, 0],
          [0, 0, 0, ..., 1, 1, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
  

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:47:41,027 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:47:41,029 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:47:41,031 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:47:41,032 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:47:41,033 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:47:51,698 - DeepRL - DEBUG - opt started...
2021-07-02 14:47:51,699 - DeepRL - DEBUG - started testing...
2021-07-02 14:50:21,342 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_standardInput/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:50:57,423 - DeepRL - DEBUG - global iter =    0 total obj: 11946.5885
2021-07-02 14:51:10,810 - DeepRL - DEBUG - global iter =    1 total obj: 11294.3618
2021-07-02 14:51:21,766 - DeepRL - DEBUG - global iter =    2 total obj: 10940.0137
2021-07-02 14:51:33,700 - DeepRL - DEBUG - global iter =    3 total obj: 10746.5794
2021-07-02 14:51:44,502 - DeepRL - DEBUG - global iter =    4 total obj: 10605.1208
2021-07-02 14:51:56,209 - DeepRL - DEBUG - global iter =    5 total obj: 10483.5052
2021-07-02 14:52:08,040 - DeepRL - DEBUG - global iter =    6 total obj: 10385.4550
2021-07-02 14:52:20,032 - DeepRL - DEBUG - global iter =    7 total obj: 10318.1457
2021-07-02 14:52:33,192 - DeepRL - DEBUG - global iter =    8 total obj: 10289.1647
2021-07-02 14:52:49,717 - DeepRL - DEBUG - global iter =    9 total obj: 10257.9758
2021-07-02 14:52:49,719 - DeepRL - DEBUG - started testing...
2021-07-02 14:55:39,282 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_standardInput/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:55:56,608 - DeepRL - DEBUG - global iter =   10 total obj: 10241.2113
2021-07-02 14:56:08,906 - DeepRL - DEBUG - global iter =   11 total obj: 10212.9942
2021-07-02 14:56:21,552 - DeepRL - DEBUG - global iter =   12 total obj: 10146.2785
2021-07-02 14:56:32,770 - DeepRL - DEBUG - global iter =   13 total obj: 10086.7501
2021-07-02 14:56:44,559 - DeepRL - DEBUG - global iter =   14 total obj: 10045.7630
2021-07-02 14:56:56,347 - DeepRL - DEBUG - global iter =   15 total obj: 9983.9607
2021-07-02 14:57:09,263 - DeepRL - DEBUG - global iter =   16 total obj: 9945.5011
2021-07-02 14:57:24,027 - DeepRL - DEBUG - global iter =   17 total obj: 9948.1545
2021-07-02 14:57:39,934 - DeepRL - DEBUG - global iter =   18 total obj: 9854.1639
2021-07-02 14:57:57,787 - DeepRL - DEBUG - global iter =   19 total obj: 9821.1227
2021-07-02 14:57:57,789 - DeepRL - DEBUG - started testing...
2021-07-02 15:00:27,920 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/fullModel_standardInput/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:05:13,605 - DeepRL - DEBUG - global iter =   30 total obj: 9597.0222
2021-07-02 15:05:25,148 - DeepRL - DEBUG - global iter =   31 total obj: 9657.6004
2021-07-02 15:05:38,759 - DeepRL - DEBUG - global iter =   32 total obj: 9572.7778
2021-07-02 15:05:53,049 - DeepRL - DEBUG - global iter =   33 total obj: 9574.2843
2021-07-02 15:06:07,516 - DeepRL - DEBUG - global iter =   34 total obj: 9781.7332
2021-07-02 15:06:19,904 - DeepRL - DEBUG - global iter =   35 total obj: 9643.7564
2021-07-02 15:06:34,359 - DeepRL - DEBUG - global iter =   36 total obj: 9573.5712
2021-07-02 15:06:48,635 - DeepRL - DEBUG - global iter =   37 total obj: 9557.0786
2021-07-02 15:07:03,151 - DeepRL - DEBUG - global iter =   38 total obj: 9638.7033
2021-07-02 15:07:17,381 - DeepRL - DEBUG - global iter =   39 total obj: 9613.3962
2021-07-02 15:07:17,382 - DeepRL - DEBUG - started testing...
2021-07-02 15:09:04,972 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/fullModel_standardInput/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:09:31,667 - DeepRL - DEBUG - global iter =   40 total obj: 9563.1750
2021-07-02 15:09:45,220 - DeepRL - DEBUG - global iter =   41 total obj: 9601.7937
2021-07-02 15:09:56,500 - DeepRL - DEBUG - global iter =   42 total obj: 9509.8373
2021-07-02 15:10:10,228 - DeepRL - DEBUG - global iter =   43 total obj: 9528.1662
2021-07-02 15:10:24,540 - DeepRL - DEBUG - global iter =   44 total obj: 9469.7349
2021-07-02 15:10:37,731 - DeepRL - DEBUG - global iter =   45 total obj: 9488.6559
2021-07-02 15:10:48,910 - DeepRL - DEBUG - global iter =   46 total obj: 9520.9436
2021-07-02 15:11:00,140 - DeepRL - DEBUG - global iter =   47 total obj: 9531.0968
2021-07-02 15:11:12,916 - DeepRL - DEBUG - global iter =   48 total obj: 9528.8500
2021-07-02 15:11:27,294 - DeepRL - DEBUG - global iter =   49 total obj: 9466.7431
2021-07-02 15:11:27,295 - DeepRL - DEBUG - opt finished.
2021-07-02 15:11:27,297 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:09,461 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/fullModel_standardInput/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
          0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
          1, 1, 1, 1]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2